In [21]:
import zipfile
import pandas as pd
import numpy as np

# data_preparation.ipynb created train.zip which has train.csv inside
zipped_data_path = "../data/clean_data/train.zip"
train_csv = "train.csv"

with zipfile.ZipFile(zipped_data_path, "r") as zip:
    train_data = pd.read_csv(zip.open(train_csv))

In [22]:
from sklearn.model_selection import train_test_split

ALL_FEATURES = ['TRIP_ID', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'MISSING_DATA',
 'POLYLINE', 'DAY_TYPE_A', 'DAY_TYPE_C', 'DAY_TYPE_B',
 'CALL_TYPE_A', 'CALL_TYPE_B', 'CALL_TYPE_C', 'YEAR_2013', 'YEAR_2014',
 'MONTH_1', 'MONTH_2', 'MONTH_3', 'MONTH_4', 'MONTH_5', 'MONTH_6',
 'MONTH_7', 'MONTH_8', 'MONTH_9', 'MONTH_10', 'MONTH_11', 'MONTH_12',
 'DAY_OF_WEEK_0', 'DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3',
 'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5', 'DAY_OF_WEEK_6', 'HOUR_0', 'HOUR_1',
 'HOUR_2', 'HOUR_3', 'HOUR_4', 'HOUR_5', 'HOUR_6', 'HOUR_7', 'HOUR_8',
 'HOUR_9', 'HOUR_10', 'HOUR_11', 'HOUR_12', 'HOUR_13', 'HOUR_14',
 'HOUR_15', 'HOUR_16', 'HOUR_17', 'HOUR_18', 'HOUR_19', 'HOUR_20',
 'HOUR_21', 'HOUR_22', 'HOUR_23']

# We could totally change this. Utilization of these just probably requires further preprocessing.
ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION = ['TRIP_ID', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'POLYLINE']

X = train_data.drop("TRAVEL_TIME", axis=1)
X = X.loc[:, ~X.columns.isin(ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION)]
y = train_data["TRAVEL_TIME"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=420)

In [23]:
from sklearn.metrics import mean_squared_error

def root_mean_squared_error(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

# This is gonna run for a really long time unless we operate on a sample 
def train_test_split_sample(frac, random_state=420):
    # TODO: We should use stratified sampling here. Would help get more representative samples.
    train_data_sample = train_data.sample(frac=frac, random_state=random_state) # frac is used to control percentage of train data used
    X_sample = train_data_sample.drop("TRAVEL_TIME", axis=1)
    X_sample = X_sample.loc[:, ~X_sample.columns.isin(ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION)]
    y_sample = train_data_sample["TRAVEL_TIME"]

    X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split(X_sample, y_sample, test_size=0.2, random_state=420)
    return X_train_sample, X_test_sample, y_train_sample, y_test_sample

### Baseline: predicting average

In [24]:
# Establish a baseline for model comparison. What if we always predict the average travel time?
avg_travel_time = y_train.mean()
y_pred = [ avg_travel_time for i in range(len(X_test)) ]

root_mean_squared_error(y_test, y_pred)

678.8930754443071

#### Reviewing Baseline:
* RMSE = 678.8930754443071
* the most naive approach

### Linear Regression

In [25]:
# Running a linear regression
from sklearn import linear_model

lreg = linear_model.LinearRegression()
lreg.fit(X_train, y_train)
y_pred = lreg.predict(X_test)

root_mean_squared_error(y_test, y_pred)

671.0856201370245

In [26]:
# Lasso
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

rmse = make_scorer(lambda y_true, y_pred: root_mean_squared_error(y_true, y_pred), greater_is_better=False)
X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split_sample(frac=0.01)

lasso = Lasso()

param_grid = {
    'alpha': np.logspace(-5, 2, 50)
}

search = GridSearchCV(lasso, param_grid, n_jobs = -1, scoring=rmse)
search.fit(X_train_sample, y_train_sample)

print(f'Best CV score: {-1 * search.best_score_}')
print(search.best_params_)

/Users/tylerholston/Desktop/151b-kaggle-competition/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.587e+08, tolerance: 4.165e+05
  model = cd_fast.enet_coordinate_descent(
/Users/tylerholston/Desktop/151b-kaggle-competition/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.151e+08, tolerance: 3.787e+05
  model = cd_fast.enet_coordinate_descent(
/Users/tylerholston/Desktop/151b-kaggle-competition/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increa

Best CV score: 593.3799185880491
{'alpha': 0.3727593720314938}


In [27]:
# Test Lasso
lasso = Lasso(alpha = 0.3727593720314938)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)

root_mean_squared_error(y_test, y_pred)

671.1443087427473

In [28]:
# Ridge
from sklearn.linear_model import Ridge

lasso = Ridge()

param_grid = {
    'alpha': np.logspace(-5, 2, 50)
}

search = GridSearchCV(lasso, param_grid, n_jobs = -1, scoring=rmse)
search.fit(X_train_sample, y_train_sample)

print(f'Best CV score: {-1 * search.best_score_}')
print(search.best_params_)

Best CV score: 593.391937030348
{'alpha': 51.79474679231202}


In [29]:
# Test Ridge
ridge = Ridge(alpha = 51.79474679231202)
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)

root_mean_squared_error(y_test, y_pred)

671.0895985241863

#### Reviewing Linear Regression:
* Linear Regression: 671.0856201370245
* Lasso Regression: 671.1443087427473
* Ridge Regression: 671.0895985241863

### Support Vector Machines (SVM)

In [23]:
# !!!! SKIP THIS IF YOU DON'T WANT TO WAIT A LONG TIME !!!!

# Running a SVM. Using 0.1% of original data runs in ~2mins on my laptop, 0.5% runs in ~23mins.
# This can be skipped unless you really want to rerun it. 0.5% yielded:
# Best CV score: 600.6364736351106
# {'C': 100.0, 'kernel': 'linear'}
# so just using this for now.


from sklearn.svm import SVR

X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split_sample(frac=0.005)

svm = SVR()

param_grid = {
    'C': np.logspace(0, 2, 25),
    'kernel': ['rbf', 'sigmoid', 'linear']
}

search = GridSearchCV(svm, param_grid, n_jobs = -1, scoring=rmse)
search.fit(X_train_sample, y_train_sample)

print(f'Best CV score: {-1 * search.best_score_}')
print(search.best_params_)

svm_poly = SVR(kernel = 'poly')

param_grid = {
    'C': np.logspace(0, 2, 25),
    'degree': np.arange(1, 4)
}

search = GridSearchCV(svm, param_grid, n_jobs = -1, scoring=rmse)
search.fit(X_train_sample, y_train_sample)

print(f'Best CV score: {-1 * search.best_score_}')
print(search.best_params_)

Best CV score: 600.6364736351106
{'C': 100.0, 'kernel': 'linear'}
Best CV score: 602.4719756396436
{'C': 56.23413251903491, 'degree': 1}


In [31]:
# Test the SVM
from sklearn.svm import SVR
svm = SVR(kernel = 'linear', C=100)

svm.fit(X_train_sample, y_train_sample)
y_pred = svm.predict(X_test_sample)

root_mean_squared_error(y_test_sample, y_pred)

637.8973953407794

In [32]:
# That was good but does it generalize to another larger sample? - No -
X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split_sample(frac=0.015)

svm = SVR(kernel = 'linear', C=100)

svm.fit(X_train_sample, y_train_sample)
y_pred = svm.predict(X_test_sample)

root_mean_squared_error(y_test_sample, y_pred)

730.0220477768397

#### Reviewing SVM:
* SVM did well on 0.5% of data w/ RSME = 637.8973953407794
* seemed to perform worse with more data (1.5%) w/ RSME = 730.0220477768397
* could try some other sample sizes on not my shitty laptop

### Bagging

In [33]:
# Bagging w/ Decision Trees
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier

X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split_sample(frac=0.05)

bag = BaggingRegressor(estimator = DecisionTreeClassifier(), random_state = 420)

param_grid = {
    'n_estimators': list(range(1,20))
}

search = GridSearchCV(bag, param_grid, n_jobs = -1, scoring = rmse)
search.fit(X_train_sample, y_train_sample)

print(f'Best CV score: {-1 * search.best_score_}')
print(search.best_params_)

/Users/tylerholston/Desktop/151b-kaggle-competition/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best CV score: 759.6824958854044
{'n_estimators': 19}


In [41]:
# Test the SVM
bag = BaggingRegressor(estimator = DecisionTreeClassifier(), random_state = 420, n_estimators=19)

bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)

root_mean_squared_error(y_test, y_pred)

743.3402218019256

#### Reviewing Bagging:
* RSME = 743.3402218019256 on all training data

### Random Forest

In [35]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor

X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split_sample(frac=0.001)

rf = RandomForestRegressor(random_state=420)

param_grid = {
    'n_estimators': list(range(10,30)),
    'criterion': ['squared_error', 'absolute_error', 'friedman_mse']
}

search = GridSearchCV(rf, param_grid, n_jobs = -1, scoring = rmse)
search.fit(X_train_sample, y_train_sample)

print(f'Best CV score: {-1 * search.best_score_}')
print(search.best_params_)

Best CV score: 647.7588569489957
{'criterion': 'friedman_mse', 'n_estimators': 24}


In [37]:
# Test the Random Forest
rf = RandomForestRegressor(random_state=420, criterion = 'friedman_mse', n_estimators = 24)

rf.fit(X_train_sample, y_train_sample)
y_pred = rf.predict(X_test_sample)

root_mean_squared_error(y_test_sample, y_pred)

709.0740712229938

In [40]:
rf = RandomForestRegressor(random_state=420, criterion = 'friedman_mse', n_estimators = 24)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

root_mean_squared_error(y_test, y_pred)

668.6428710752018

#### Reviewing Random Forest:
* RSME = 668.6428710752018 on all training data
* solid baseline to try and beat with deep learning